In [1]:
import pandas as pd



liste_dico = [{"comp_id" : 549,
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"comp_id" : 393,
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"comp_id" : 243,
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]

In [2]:
i = 0

dico = liste_dico[i]

data_import = pd.read_excel(f"../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/data_physical.xlsx", index_col = 0)


In [13]:
data = data_import.set_index("team_name").fillna(0)

nb_matchs = pd.Series(index = data.index.unique())
for team in nb_matchs.index :
    nb_matchs[team] = len(data.loc[team].match_id.unique())
nb_matchs = nb_matchs.reindex(dico["ranking"])

drop = ["player_name", "player_short_name", "player_id", "player_birthdate", "team_id", "match_name", "match_date", "competition_name", "competition_id", "season_name",
        "season_id", "competition_edition_id", "position", "position_group", "minutes_full_tip", "minutes_full_otip", "physical_check_passed"]

data.drop(drop, inplace = True, axis = 1)

data = data.groupby(["team_name", "match_id"]).sum()

nb_minute_match = data.pop("minutes_full_all")
data = data.multiply(900/nb_minute_match, axis = 0)

data = data.reset_index().drop("match_id", axis = 1).groupby("team_name", as_index = True).sum().reindex(dico["ranking"])

data = data.divide(nb_matchs, axis = 0)

top5 = dico["ranking"][:5]
top15 = dico["ranking"][5:]
top5_df = data.loc[top5]
top15_df = data.loc[top15]

df_final = pd.DataFrame(index = top5_df.columns)

df_final["Moyenne Top 5"] = top5_df.mean(axis = 0)
df_final["Moyenne Bottom 15"] = top15_df.mean(axis = 0)

df_final["Diff. Top 5 avec Bottom 15 en %"] = (100*(df_final["Moyenne Top 5"] - df_final["Moyenne Bottom 15"])/abs(df_final["Moyenne Bottom 15"])).round(2)

df_final["Ecart type Top 5"] = top5_df.std(axis = 0)
df_final["Ecart type Bottom 15"] = top15_df.std(axis = 0)

df_final["Min Top 5"] = top5_df.min(axis = 0)
df_final["Min Bottom 15"] = top15_df.min(axis = 0)

df_final["Max Top 5"] = top5_df.max(axis = 0)
df_final["Max Bottom 15"] = top15_df.max(axis = 0)

df_final = df_final.reindex(abs(df_final).sort_values(by = "Diff. Top 5 avec Bottom 15 en %", ascending = False).index)

In [14]:
data

total_distance_full_all  \
team_name       match_id                            
AC Ajaccio      1007270                  112074.0   
                1021412                  112309.0   
                1041033                  109286.0   
                1050077                  119101.0   
                1065100                  108611.0   
...                                           ...   
Valenciennes FC 1497641                  118007.0   
                1507318                  114518.0   
                1514140                  109681.0   
                1527196                  111524.0   
                1539377                  119161.0   

                          total_metersperminute_full_all  \
team_name       match_id                                   
AC Ajaccio      1007270                          1527.33   
                1021412                          1699.53   
                1041033                          1552.54   
                1050077                          1871.08   
                1065100                          1757.91   
...                                                  ...   
Valenciennes FC 1497641                          1880.33   
                1507318                          1832.54   
                1514140                          1551.72   
                1527196                          1653.75   
                1539377                          1665.67   

                          running_distance_full_all  hsr_distance_full_all  \
team_name       match_id                                                     
AC Ajaccio      1007270                     15068.0                 5873.0   
                1021412                     14616.0                 4696.0   
                1041033                     13656.0                 5060.0   
                1050077                     16769.0                 6056.0   
                1065100                     13890.0                 5170.0   
...                                             ...                    ...   
Valenciennes FC 1497641                     17139.0                 5677.0   
                1507318                     18056.0                 6549.0   
                1514140                     16218.0                 5209.0   
                1527196                     16190.0                 5524.0   
                1539377                     20011.0                 7203.0   

                          hsr_count_full_all  sprint_distance_full_all  \
team_name       match_id                                                 
AC Ajaccio      1007270                604.0                    2242.0   
                1021412                523.0                    1199.0   
                1041033                480.0                    1290.0   
                1050077                567.0                    1507.0   
                1065100                504.0                    1458.0   
...                                      ...                       ...   
Valenciennes FC 1497641                565.0                    1481.0   
                1507318                596.0                    1888.0   
                1514140                503.0                    1240.0   
                1527196                507.0                    1687.0   
                1539377                624.0                    2145.0   

                          sprint_count_full_all  hi_distance_full_all  \
team_name       match_id                                                
AC Ajaccio      1007270                   124.0                8115.0   
                1021412                    81.0                5895.0   
                1041033                    77.0                6350.0   
                1050077                    94.0                7563.0   
                1065100                    90.0                6628.0   
...                                         ...                   

In [12]:
data

total_distance_full_all  \
team_name       match_id                            
AC Ajaccio      1007270              96366.294067   
                1021412              99682.544379   
                1041033              98981.976270   
                1050077             110257.151380   
                1065100             102835.095471   
...                                           ...   
Valenciennes FC 1497641             111936.320233   
                1507318             108170.779064   
                1514140              99179.041495   
                1527196             102820.791248   
                1539377             107729.683576   

                          total_metersperminute_full_all  \
team_name       match_id                                   
AC Ajaccio      1007270                      1313.267412   
                1021412                      1508.458580   
                1041033                      1406.158862   
                1050077                      1732.142894   
                1065100                      1664.424807   
...                                                  ...   
Valenciennes FC 1497641                      1783.599456   
                1507318                      1730.970498   
                1514140                      1403.142771   
                1527196                      1524.693192   
                1539377                      1505.879458   

                          running_distance_full_all  hsr_distance_full_all  \
team_name       match_id                                                     
AC Ajaccio      1007270                12956.147893            5049.871023   
                1021412                12972.781065            4168.047337   
                1041033                12368.444887            4582.918214   
                1050077                15523.817361            5606.311523   
                1065100                13151.333439            4895.060754   
...                                             ...                    ...   
Valenciennes FC 1497641                16257.311791            5384.955892   
                1507318                17055.236616            6186.018199   
                1514140                14665.126093            4710.238119   
                1527196                14926.550431            5092.913192   
                1539377                18091.310899            6512.004018   

                          hsr_count_full_all  sprint_distance_full_all  \
team_name       match_id                                                 
AC Ajaccio      1007270           519.346518               1927.773001   
                1021412           464.201183               1064.201183   
                1041033           434.743230               1168.372430   
                1050077           524.897397               1395.097666   
                1065100           477.197412               1380.463942   
...                                      ...                       ...   
Valenciennes FC 1497641           535.934486               1404.812344   
                1507318           562.966384               1783.356598   
                1514140           454.837737               1121.269969   
                1527196           467.434285               1555.348399   
                1539377           564.138624               1939.226519   

                          sprint_count_full_all  hi_distance_full_all  \
team_name       match_id                                                
AC Ajaccio      1007270              106.620808           6977.644024   
                1021412               71.893491           5232.248521   
                1041033               69.740060           5751.290644   
                1050077               87.020027           7001.409190   
                1065100               85.213824           6275.524696   
...                                         ...                   